# Lecture 3

## Preventive maintainence

Harold Zurcher's problem: The state $x\in\mathcal{X}=\left\{x_{0},...,\bar{x}\right\}$ of each bus at each period $t$ is the mileage since last overhaul. The transition between states is as follows:
* When no overhaul is performed, these states undergo random transitions (depending on how much the bus is used): $x_{i}\rightarrow x_{i^{\prime}}$ with some probability $P_{i^{\prime}|i}$, where $i^{\prime}\geq i$.

* When overhaul is performed on a bus, the state is restored to the zero state $x_{0}$.

There is a fixed cost $C$ associated with overhaul (independent of mileage), while operations costs $c\left(  x\right)$ increase with mileage (maintenance, fuel, insurance and costs of nexpected breakdowns).

## Translating the problem

Assume the states are discretized into $12,500$ mile brackets.

There are $30$ states, so $\mathcal{X}=\left\{  1,...,30\right\}$: `nbX = 30`.

The choice set is $\mathcal{Y}=\left\{  y_{0}=1,y_{1}=2\right\}$ (operate or overhaul): `nbY = 2`.

There are $40$ periods (quarter years over $10$ years): `nbT = 40`.

In [ ]:
library("Matrix")
library("gurobi")
nbX = 30  #30
nbT = 40  #40
nbY = 2  # choice set: 1=run as usual; 2=overhaul

#### Transitions:
* If no overhaul is performed, each state but the last one has a probability $25\%$ of transiting to the next one, and probability $75\%$ of remaining the same. The last state transits to $1$ with probability $25\%$ (overhaul is performed when beyond last state).

* If overhaul is performed, the state transits to $1$ for sure.

We are going to solve the dual problem 
\begin{align*}
\min_{U}  & \, b^{\intercal}U\\
s.t.~  &  AU\geq u~\left[  \pi\right] 
\end{align*}

First let's construct our constraint matrix $A$.

We build the transition matrix $P_{x^{\prime}|xy}$ matrix of dimension `nbXnbY`$\times$ `nbX`

Let
\begin{align*}
L_{\mathcal{X}}=%
\begin{pmatrix}
0 & 1 & 0 & 0\\
0 & \ddots & \ddots & 0\\
0 & \ddots & \ddots & 1\\
1 & 0 & 0 & 0
\end{pmatrix}
\text{ and }R_{\mathcal{X}}=%
\begin{pmatrix}
1 & 0 & \cdots & 0\\
1 & \vdots & \ddots  & \vdots\\
1 & \vdots &  \ddots & \vdots\\
1 & 0 & \cdots & 0
\end{pmatrix}
\end{align*}
so that $P$ is given by
\begin{align*}
P=1_{y_{0}}\otimes\left(  0.75I_{\mathcal{X}}+0.25L_{\mathcal{X}}\right)
+1_{y_{1}}\otimes R_{\mathcal{X}}%
\end{align*}

Which in R looks like

In [ ]:
IdX = sparseMatrix(1:nbX, 1:nbX)
LX = sparseMatrix(c(nbX, 1:(nbX - 1)), 1:nbX)
RX = sparseMatrix(1:nbX, rep(1, nbX), dims = c(nbX, nbX))
P = kronecker(c(1, 0), 0.75 * IdX + 0.25 * LX) + kronecker(c(0, 1), RX)

Recall the constraint matrix $A$ can be expressed as
\begin{align*}
A &= I_{\mathcal{T}}\otimes J-N_{\mathcal{T}}\otimes P \\
  &= I_{\mathcal{T}} \otimes1_{\mathcal{Y}}\otimes I_{\mathcal{X}}-N_{\mathcal{T}}\otimes P.
\end{align*}
where
\begin{align*}
N_{\mathcal{T}}=
\begin{pmatrix}
0 & 1 & 0 & \cdots & 0\\
\vdots & \ddots & \ddots &  & \vdots\\
&  & \ddots & \ddots & 0\\
\vdots &  &  & \ddots & 1\\
0 & \cdots &  & \cdots & 0
\end{pmatrix}
\end{align*}

In [ ]:
IdT = sparseMatrix(1:nbT, 1:nbT)
NT = sparseMatrix(1:(nbT - 1), 2:nbT, dims = c(nbT, nbT))
A = kronecker(kronecker(IdT, matrix(1, nbY, 1)), IdX) - kronecker(NT, P)

#### Costs:
* The cost of replacing an engine is $C=\$8,000$ (in $1985$ dollars).

* The operations cost is $c\left(  x\right)  =x\times5.10^{2}.$

The discount factor is $\beta=0.9$.

In [ ]:
overhaulCost = 8000
maintCost = function(x) (x * 500)
beta = 0.9

Next, we build $u_{xyt}$ 
* First the $u_{xy}$'s so that $u_{x1}=-x\times5.10^{2}$ for $x<\bar{x}$, and $u_{\bar{x}1}=-C$, while $u_{x2}=-C$ for all $x$.

* Next the $u_{xyt}$ so that $u_{xyt}=u_{xy}\beta^{t}=vec\left(\left(\beta^{1},...,\beta^{T}\right)  \otimes u_{xy}\right)$

Finially we build $b_{xt}$

In [ ]:
n1_x = rep(1, nbX)
u_xy = c(-maintCost(1:(nbX - 1)), rep(-overhaulCost, nbX + 1))
u_xyt = c(kronecker(beta^(1:nbT), u_xy))

b_xt = c(n1_x, rep(0, nbX * (nbT - 1)))

result = gurobi(list(A = A, obj = c(b_xt), modelsense = "min", rhs = u_xyt, sense = ">", 
    lb = -Inf), params = list(OutputFlag = 0))

U_x_t_gurobi = array(result$x, dim = c(nbX, nbT))

print(U_x_t_gurobi[, 1])

## Backward induction

The smarter way to approach this problem is of course using backwards induction

In [ ]:
U_x_t = matrix(0, nbX, nbT)
contVals = apply(X = array(u_xyt, dim = c(nbX, nbY, nbT))[, , nbT], FUN = max, MARGIN = 1)
U_x_t[, nbT] = contVals

for (t in (nbT - 1):1) {
    myopic = array(u_xyt, dim = c(nbX, nbY, nbT))[, , t]
    Econtvals = matrix(P %*% contVals, nrow = nbX)
    
    contVals = apply(X = myopic + Econtvals, FUN = max, MARGIN = 1)
    U_x_t[, t] = contVals
}

Which give identical solutions to using LP

In [ ]:
print(U_x_t[,1])